# CLOE tester

This notebook is built to test that everything works smoothly in CLOE, through a comparative analysis between the fiducial spectra and the data to assess the implementation of the Weyl bypass, along with carrying out a grid likelihood test on a selected parameter.

**WARNING:** this is a version built for TWG WP11. It does not allow to change the case in which the test is done and only focuses on what has been chosen by ISTL as the baseline

In [ ]:
import sys,os
import numpy             as np
import matplotlib.pyplot as plt
import pandas            as pdo
import seaborn           as sb

from itertools import product
from copy      import deepcopy
from time      import time
from tqdm      import tqdm
from shutil    import rmtree

likelihood_path = os.path.realpath(os.path.join(os.getcwd(),'..'))
sys.path.insert(0, likelihood_path)
print('Setting as working directory: ', likelihood_path)

import warnings
warnings.filterwarnings('ignore')

from cobaya.model import get_model
from cobaya.run   import run

from cloe.cobaya_interface             import EuclidLikelihood
from cloe.photometric_survey.photo     import Photo
from cloe.data_reader.reader           import Reader
from cloe.user_interface.likelihood_ui import LikelihoodUI
from cloe.auxiliary                    import likelihood_yaml_handler as lyh

import matplotlib
from matplotlib import rc
from matplotlib.pyplot import cm
rc('text', usetex=True)
rc('font', family='serif')
matplotlib.rcParams.update({'font.size': 18})

red    = '#8e001c'
yellow = '#ffb302'

sidelegend = {'bbox_to_anchor': (1.04,0.5), 
              'loc': "center left",
              'frameon': False}

here = os.path.abspath(os.getcwd())+'/'

import pprint
pp = pprint.PrettyPrinter(indent=3)

# Test settings

Choose what to run here

- varpar: choose for which parameter to run the grid test **Warning:** currently working only with primary parameters;
- Ngrid: how many elements to include in the grid;
- perc_lim: percentage shift from the fiducial to use as the extremes of the grid (e.g. $\pm 1\%$);
- observables: dictionary containing the probe combinations to run.

In [ ]:
varpar        = 'H0'
Ngrid         = 8
perc_lim      = 1


observables = {'WL': {'WL': True,
                      'GCphot': True,
                      'GCspectro': False},
               'GCphot': {'GCphot': True,
                          'GCspectro': False},
               'GCspectro': {'GCspectro': False},
               'CG': {'CG': False},
               'add_phot_RSD': False,
               'matrix_transform_phot' : False}


theory_dict    = {'camb': {'extra_args': {'halofit_version': 'mead',
                                          'nnu': 3.046,
                                          'dark_energy_model': 'ppf',
                                          'num_massive_neutrinos': 1}}}



labels ={'w': r'$w_0$',
         'wa': r'$w_a$',
         'ns': r'$n_s$',
         'mnu': r'$m_\nu$',
         'nnu': r'$N_\nu$',
         'tau': r'$\tau$',
         'H0': r'$H_0$',
         'ombh2': r'$\Omega_b\,h^2$',
         'omk': r'$\Omega_k$',
         'omch2': r'$\Omega_c\,h^2$',
         'logA': r'$\log{A_s}$'}

# Fiducial, flags and specs

Here we read the dictionary containing the settings

**NOTE:** The input_files folder containing the data is required.

In [ ]:
reference_dict = np.load('./input_files/reference_settings.npy',allow_pickle=True).item()

reference_dict['params']['As'] = np.exp(reference_dict['params'].pop('logA'))*1.e-10

fiducial = reference_dict['params']
Nbins    = reference_dict['specs']['Nbins']

pp.pprint(reference_dict)
    
pargrid = {varpar: np.linspace(fiducial[varpar]*(1-perc_lim/100),
                               fiducial[varpar]*(1+perc_lim/100),
                               Ngrid)}

# Defining fiducial info dictionary

This sets up the dictionary to be passed to Cobaya to use CLOE. Only includes photo survey

In [ ]:
default = LikelihoodUI()
    
cobaya_dict = default._config['Cobaya']
likelihood_euclid_dict = cobaya_dict['likelihood']['Euclid']

default._check_and_update_likelihood_fields(likelihood_euclid_dict)
default._check_and_update_params_field(cobaya_dict)

cobaya_dict['likelihood']['Euclid']['data']['photo']['luminosity_ratio'] = here+'input_files/luminosity_ratio.dat'
cobaya_dict['likelihood']['Euclid']['data']['photo']['IA_model'] = 'eNLA'
cobaya_dict['likelihood']['Euclid']['data']['photo']['cov_3x2pt'] = \
here+'input_files/CovMat-3x2pt-Gauss-32Bins.npz'
cobaya_dict['likelihood']['Euclid']['data']['photo']['cov_GC'] = 'CovMat-PosPos-{:s}-32Bins.npz'
cobaya_dict['likelihood']['Euclid']['data']['photo']['cov_WL'] = 'CovMat-ShearShear-{:s}-32Bins.npz'
cobaya_dict['likelihood']['Euclid']['data']['photo']['cov_model'] = 'Gauss'
cobaya_dict['likelihood']['Euclid']['data']['photo']['cov_is_num'] = False
cobaya_dict['likelihood']['Euclid']['data']['photo']['cov_nsim'] = 10000
cobaya_dict['likelihood']['Euclid']['data']['photo']['ndens_GC'] = here+'input_files/nzTabSPV3.dat'
cobaya_dict['likelihood']['Euclid']['data']['photo']['ndens_WL'] = here+'input_files/nzTabSPV3.dat'
cobaya_dict['likelihood']['Euclid']['data']['photo']['root_GC'] = here+'input_files/Cls_zNLA_PosPos_C00.dat'
cobaya_dict['likelihood']['Euclid']['data']['photo']['root_WL'] = here+'input_files/Cls_zNLA_ShearShear_C00.dat'
cobaya_dict['likelihood']['Euclid']['data']['photo']['root_XC'] = here+'input_files/Cls_zNLA_PosShear_C00.dat'
cobaya_dict['likelihood']['Euclid']['data']['photo']['Fourier'] = True
cobaya_dict['likelihood']['Euclid']['data']['photo']['redshifts'] = reference_dict['redshifts']





cobaya_dict['likelihood']['Euclid']['data']['sample'] = 'ExternalBenchmark'


cobaya_dict['theory'] = deepcopy(theory_dict)
cobaya_dict['params'] = deepcopy(reference_dict['params']) 


cobaya_dict['likelihood']['Euclid']['NL_flag_phot_matter']   = 3
cobaya_dict['likelihood']['Euclid']['NL_flag_phot_baryon']   = 2
cobaya_dict['likelihood']['Euclid']['add_phot_RSD']          = reference_dict['flags']['add_phot_RSD']
cobaya_dict['likelihood']['Euclid']['bias_model']            = reference_dict['flags']['bias_model']
cobaya_dict['likelihood']['Euclid']['magbias_model']         = reference_dict['flags']['magbias_model']
cobaya_dict['likelihood']['Euclid']['matrix_transform_phot'] = False
cobaya_dict['likelihood']['Euclid']['use_gamma_MG']          = False
cobaya_dict['likelihood']['Euclid']['use_Weyl']              = False
cobaya_dict['likelihood']['Euclid']['statistics_photo']      = 'angular_power_spectrum'

# cobaya_dict['params']['b0_mag_poly'] = 0
# cobaya_dict['params']['b1_mag_poly'] = 0
# cobaya_dict['params']['b2_mag_poly'] = 0
# cobaya_dict['params']['b3_mag_poly'] = 0


for bin_i in range(1,Nbins+1):
    for bin_j in range(1,Nbins+1):
        if bin_i == bin_j:
            cobaya_dict['likelihood']['Euclid']['observables_specifications']['WL'] \
            ['bins']['n{}'.format(bin_i)] = \
            {'n{}'.format(bin_j): {'ell_range': [[10, 5000]]} for bin_j in range(1,Nbins+1)}

            cobaya_dict['likelihood']['Euclid']['observables_specifications']['WL-GCphot'] \
            ['bins']['n{}'.format(bin_i)] = \
            {'n{}'.format(bin_j): {'ell_range': [[10, 5000]]} for bin_j in range(1,Nbins+1)}

            cobaya_dict['likelihood']['Euclid']['observables_specifications']['GCphot'] \
            ['bins']['n{}'.format(bin_i)] = \
            {'n{}'.format(bin_j): {'ell_range': [[10, 5000]]} for bin_j in range(1,Nbins+1)}
        else:
            cobaya_dict['likelihood']['Euclid']['observables_specifications']['WL'] \
            ['bins']['n{}'.format(bin_i)] = \
            {'n{}'.format(bin_j): {'ell_range': [[10, 5000]]} for bin_j in range(1,Nbins+1)}

            cobaya_dict['likelihood']['Euclid']['observables_specifications']['WL-GCphot'] \
            ['bins']['n{}'.format(bin_i)] = \
            {'n{}'.format(bin_j): {'ell_range': [[10, 5000]]} for bin_j in range(1,Nbins+1)}

            cobaya_dict['likelihood']['Euclid']['observables_specifications']['GCphot'] \
            ['bins']['n{}'.format(bin_i)] = \
            {'n{}'.format(bin_j): {'ell_range': [[10, 5000]]} for bin_j in range(1,Nbins+1)}
    
##########GCspectro choices to match data#########
for sp_bin in range(1,5):
    cobaya_dict['likelihood']['Euclid']['observables_specifications']['GCspectro']['multipole_power_spectrum']['bins']\
    [f'n{sp_bin}'][f'n{sp_bin}']['multipoles'][0]['krange'] = [[0.01,0.13474]]
    cobaya_dict['likelihood']['Euclid']['observables_specifications']['GCspectro']['multipole_power_spectrum']['bins']\
    [f'n{sp_bin}'][f'n{sp_bin}']['multipoles'][2]['krange'] = [[0.01,0.13474]]
    cobaya_dict['likelihood']['Euclid']['observables_specifications']['GCspectro']['multipole_power_spectrum']['bins']\
    [f'n{sp_bin}'][f'n{sp_bin}']['multipoles'][4]['krange'] = [[0.01,0.13474]]

lyh.update_cobaya_dict_with_halofit_version(cobaya_dict)

cobaya_dict['debug'] = False

cobaya_dict['likelihood']['Euclid']['observables_selection'] = {'WL': {'WL': True,
                                                                       'GCphot': True,
                                                                       'GCspectro': False},
                                                                'GCphot': {'GCphot': True,
                                                                           'GCspectro': False},
                                                                'GCspectro': {'GCspectro': True},
                                                                'CG': {'CG': False}}

del cobaya_dict['output']

# Comparing fiducial spectra with data


Here we obtain spectra from CLOE and compare them to the dataset to check for visible problems.

## CLOE engine

Extracts observables from CLOE.

One can edit this adding the flag that switches to the modified calculations of TWG WP11.

In [ ]:
info = deepcopy(cobaya_dict)
     
# info['likelihood']['Euclid']['use_Weyl'] = True   # Activating the use_Weyl flag 
    
model = get_model(info)

logposterior = model.logposterior({})
like = EuclidLikelihood()
like.initialize()
like.passing_requirements(model, info, **model.provider.params)
like.cosmo.cosmo_dic['bias_model'] = info['likelihood']['Euclid']['bias_model']
like.cosmo.cosmo_dic['magbias_model'] = info['likelihood']['Euclid']['magbias_model']


like.cosmo.cosmo_dic['redshift_bins_means_phot'] = info['likelihood']['Euclid']['data']['photo']['redshifts']
like.cosmo.update_cosmo_dic(like.cosmo.cosmo_dic['z_win'], 0.05)

read_data = Reader(info['likelihood']['Euclid']['data'])
read_data.compute_nz()
read_data.compute_luminosity_ratio()
read_data.read_phot()


phot_ins = Photo(like.cosmo.cosmo_dic,read_data.nz_dict_WL,read_data.nz_dict_GC_Phot,
                 add_RSD=info['likelihood']['Euclid']['add_phot_RSD'])

photo_obs = {'GCph': phot_ins.Cl_GC_phot,
             'WL': phot_ins.Cl_WL,
             'XC': phot_ins.Cl_cross}

## Plotting photo fiducial spectra

In [ ]:
colors = cm.rainbow(np.linspace(0, 1, Nbins))

fig, axes = plt.subplots(ncols=3, sharey=False,sharex=False,subplot_kw=dict(frameon=True),figsize=(10,8))
plt.subplots_adjust(hspace=0, wspace=0.4)

for ind,obs in enumerate(list(photo_obs.keys())):
    axes[ind].set_title(obs,loc='right',fontdict={'fontsize':16})
    for color,bin1 in zip(colors,range(1,Nbins+1)):
        cloe = photo_obs[obs](read_data.data_dict['WL']['ells'],bin1,bin1)
        
        axes[ind].plot(read_data.data_dict['WL']['ells'],cloe,label=r'$i={}$'.format(bin1),color=color)
        if obs == 'WL':
            elldata = read_data.data_dict[obs]['ells']
            data    = read_data.data_dict[obs]['E{}-E{}'.format(bin1,bin1)]
        elif obs == 'GCph':
            elldata = read_data.data_dict['GC-Phot']['ells']
            data    = read_data.data_dict['GC-Phot']['P{}-P{}'.format(bin1,bin1)]
        elif obs == 'XC':
            elldata = read_data.data_dict['XC-Phot']['ells']
            data    = read_data.data_dict['XC-Phot']['P{}-E{}'.format(bin1,bin1)]
        axes[ind].scatter(elldata,data,ls=':',color=color)
    axes[ind].ticklabel_format(axis='y', style='sci', scilimits=(0,0))
    axes[ind].set_xlabel(r'$\ell$')
    axes[ind].set_xscale('log')
    axes[ind].set_xlim([10,5000])
    if ind != 2:
        axes[ind].set_yscale('log')
axes[-1].legend(**sidelegend)
axes[0].set_ylabel(r'$C_\ell^{ixi}$');

# Test fiducial values

Here we take the default cobaya_dict defined above and copy it into an info dictionary in which we can overwrite what we want (e.g. the free parameters in this case)

This could be where the Weyl modification flag can be set

In [ ]:
import pandas as pd
fiducial_std = pd.DataFrame(columns=[varpar,'chi2','logpost'])

info = deepcopy(cobaya_dict)

info['params'][varpar] = {'prior': {'min': pargrid[varpar][0], 'max': pargrid[varpar][-1]}}

# info['likelihood']['Euclid']['use_Weyl'] = True   # Activating the use_Weyl flag 
        
info['likelihood']['Euclid']['observables_selection'] = observables

info['sampler'] = {'evaluate': {'override': {par: fiducial[par] for par in pargrid}}}

updated_info,sampler = run(info)

fiducial_std[varpar]        = np.array([fiducial[varpar]])
fiducial_std['chi2']        = np.array([-2*sampler.logposterior.loglike])
fiducial_std['logpost']     = np.array([sampler.logposterior.logpost])

# Obtaining $\chi^2$ on a grid

Same thing as above, but now we do it changing the free parameter in a grid of values. This could be done also setting the Weyl modification

In [ ]:
import pandas as pd
grid_chi2 = pd.DataFrame(columns=[varpar,'chi2','logpost'])    

info = deepcopy(cobaya_dict)

info['params'][varpar] = {'prior': {'min': pargrid[varpar][0], 'max': pargrid[varpar][-1]}}
info['likelihood']['Euclid']['observables_selection'] = observables

# info['likelihood']['Euclid']['use_Weyl'] = True   # Activating the use_Weyl flag 

par     = []
chi2 = []
logpost = []
temp = {}

for ind, vals in tqdm(enumerate(pargrid[varpar][0:2], start=0)):
    temp[varpar] = vals
    info['sampler'] = {'evaluate': {'override': temp}}
    
    updated_info,sampler = run(info)
    

    par.append(vals)
    chi2.append(-2*sampler.logposterior.loglike)
    logpost.append(sampler.logposterior.logpost)
    
grid_chi2[varpar]        = np.array(par)
grid_chi2['chi2']        = np.array(chi2)
grid_chi2['logpost']     = np.array(logpost)

grid_chi2 = pd.concat([grid_chi2])
grid_chi2.columns = ['$H_0$', '$\chi^2$', '$\log{P}$'] 

In [ ]:
plt.figure()
# Increase the linewidth for the first plot
sb.lineplot(grid_chi2, x='$H_0$', y='$\chi^2$', label='Standard CLOE', linewidth=3, color = 'black')
# Custom dash pattern for the second plot: longer dash, longer space
plt.xlabel(labels[varpar])
plt.axvline(x=fiducial[varpar], color='gray', ls=':', label='Fiducial')
plt.ylabel(r'$\chi^2$')
plt.legend(**sidelegend);
plt.show()